In [1]:
import xarray as xr
import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
from shapely.geometry import Polygon, MultiPolygon, MultiPoint
import matplotlib.pyplot as plt

import geokit as gk

import FINE as fn
import FINE.spagat.dataset as spd
import FINE.spagat.representation as spr 
import FINE.IOManagement.xarray_io as xrio 

# FUNCTIONS 

In [2]:
def read_shape_data(sds, SHP_PATH, index_col='e-id', geometry_column='geometry', srs=4326):
    """Read shape data from shapefiles."""

    #USING GEOKIT
    gkdf = gk.vector.extractFeatures(SHP_PATH, srs=srs)
    region_ids = gkdf[index_col].values

    sds.gkdf = gkdf
    sds.add_region_data(region_ids)
    sds.add_objects(description ='gk_geometries',  
                    dimension_list =['region_ids'], 
                    object_list = gkdf.geom)


    # USING GEOPANDAS 
#     gdf = gpd.read_file(SHP_PATH)
# region_ids = gdf[index_col].values
#     sds.gdf = gdf
#     sds.add_region_data(region_ids)
#     sds.add_objects(description ='gpd_geometries',  
#                     dimension_list =['region_ids'], 
#                     object_list = gdf.geometry)

    return sds

In [3]:
def rasterize_xarray(sds, 
                     shape, 
                     x_coord='x', 
                     y_coord='y', 
                     srs=gk.srs.EPSG3035):
    '''Takes a gridded xarray dataset and a geokit shape to raster the dataset
    based on the shape'''

    x_min = float(sds[x_coord].min().values)
    x_max = float(sds[x_coord].max().values)
    y_min = float(sds[y_coord].min().values)
    y_max = float(sds[y_coord].max().values)

    dy = sds[y_coord][1].values - sds[y_coord][0].values
    dx = sds[x_coord][1].values - sds[x_coord][0].values

    # TODO: discuss with Sev whether the following can be generalized
    # without using dx and dy, such that generalization for cosmo becomes easier
    gk_ds = gk.vector.rasterize(source=shape, 
                                pixelWidth=dx, 
                                pixelHeight=dy,
                                srs=srs, 
                                bounds=(x_min, y_min, x_max, y_max))

    # TODO: understand, why y axis is inversed when ReadAsArray
    return np.flip(gk_ds.ReadAsArray(), 0)
    # return gk_ds.ReadAsArray()


In [4]:
def add_rasters(sds, 
                region_description='region_ids',
                x_coord='x', 
                y_coord='y',
                srs=gk.srs.EPSG3035):

        sds.xr_dataset['rasters'] = ([region_description, 'y', 'x'],
                                      [rasterize_xarray(sds.xr_dataset, 
                                                            geometry.item(0),
                                                            x_coord=x_coord, 
                                                            y_coord=y_coord,
                                                            srs=srs)
                                       for geometry in sds.xr_dataset['gk_geometries']])
